<a href="https://colab.research.google.com/github/megnaanand/climate-change-analysis/blob/main/climate_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. Introduction

This data exploration and inference project aims to uncover correlations between environmental changes and a global rise in temperatures over the years 1880 to 2020, utilizing datasets provided by NASA Climate Change.

### 2. Data Collection and Preprocessing
 We will be using  information about global Carbon Dioxide and Methane levels, Global Temperature, Ocean Warming, Ice Sheets Sea Levels, and Arctic Sea Ice levels.
